In [1]:
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"
# https://github.com/tensorflow/tensorflow/issues/33721 

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers
import seaborn as sns
import time
from csv import QUOTE_NONE

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
train_df = pd.read_csv('../sts-train.csv', sep='\t', usecols=[4, 5, 6], header=None, quoting=QUOTE_NONE, names=['score', 'sentence1', 'sentence2'])
validation_df = pd.read_csv('../sts-dev.csv', sep='\t', usecols=[4, 5, 6], header=None, quoting=QUOTE_NONE, names=['score', 'sentence1', 'sentence2'])
test_df = pd.read_csv('../sts-test.csv', sep='\t', usecols=[4, 5, 6], header=None, quoting=QUOTE_NONE, names=['score', 'sentence1', 'sentence2'])

In [5]:
train_df.head()

,score,sentence1,sentence2
0,5.00,A plane is taking off.,An air plane is taking off.
1,3.80,A man is playing a large flute.,A man is playing a flute.
2,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,2.60,Three men are playing chess.,Two men are playing chess.
4,4.25,A man is playing the cello.,A man seated is playing the cello.


In [6]:
train_df = train_df.dropna()
validation_df = validation_df.dropna()
test_df = test_df.dropna()

In [7]:
train_df.count()

score        5749
sentence1    5749
sentence2    5749
dtype: int64

In [8]:
validation_df.count()

score        1500
sentence1    1500
sentence2    1500
dtype: int64

In [9]:
test_df.count()

score        1379
sentence1    1379
sentence2    1379
dtype: int64

# Data Generator

In [10]:
max_length = 128  # Maximum length of input sentence to the model.
batch_size = 32
epochs = 50

In [11]:
class AlbertSemanticDataGenerator(tf.keras.utils.Sequence):
    def __init__(
        self,
        sentence_pairs,
        scores,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.scores = scores
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.AlbertTokenizer.from_pretrained(
            "albert-base-v2", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            truncation=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            scores = np.array(self.scores[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], scores
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

# Setup Model Architecture

In [12]:
# Create the model under a distribution strategy scope.
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    albert_model = transformers.TFAlbertModel.from_pretrained("albert-base-v2")
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    albert_model.trainable = False

    albert_outputs = albert_model(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    last_hidden_state = albert_outputs[0]
    
    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(last_hidden_state)
    dense_layer = tf.keras.layers.Dense(32, activation="relu")(avg_pool)
    dropout = tf.keras.layers.Dropout(0.5)(dense_layer)
    output_layer = tf.keras.layers.Dense(1)(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output_layer
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="mse",
        metrics=[
            tf.keras.metrics.RootMeanSquaredError()
        ],
    )


print(f"Strategy: {strategy}")
model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Some layers from the model checkpoint at albert-base-v2 were not used when initializing TFAlbertModel: ['predictions']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFAlbertModel were initialized from the model checkpoint at albert-base-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task

# Convert Data

In [13]:
train_data = AlbertSemanticDataGenerator(
    train_df[["sentence1", "sentence2"]].values.astype("str"),
    train_df['score'].values,
    batch_size=batch_size,
    shuffle=True,
)

valid_data = AlbertSemanticDataGenerator(
    validation_df[["sentence1", "sentence2"]].values.astype("str"),
    validation_df['score'].values,
    batch_size=batch_size,
    shuffle=False,
)

test_data = AlbertSemanticDataGenerator(
    test_df[["sentence1", "sentence2"]].values.astype("str"),
    test_df['score'].values,
    batch_size=batch_size,
    shuffle=False,
)

# Train Model without Fine-tuning

In [14]:
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [15]:
time_callback = TimeHistory()

In [16]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
    callbacks = [time_callback]
)

Epoch 1/50


/opt/spack/opt/spack/linux-centos8-skylake_avx512/gcc-8.3.1/python-3.8.6-57rsjc5dqlwimr7jmfiqauov4l2hsuop/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2105: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
179/179 [==============================] - 47s 214ms/step - loss: 1.8451 - root_mean_squared_error: 1.3583 - val_loss: 1.0082 - val_root_mean_squared_error: 1.0041
Epoch 2/50
179/179 [==============================] - 36s 201ms/step - loss: 1.4156 - root_mean_squared_error: 1.1898 - val_loss: 0.8845 - val_root_mean_squared_error: 0.9405
Epoch 3/50
179/179 [==============================] - 37s 204ms/step - loss: 1.2828 - root_mean_squared_error: 1.1326 - val_loss: 0.8062 - val_root_mean_squared_error: 0.8979
Epoch 4/50
179/179 [==============================] - 37s 205ms/step - loss: 1.2081 - root_mean_squared_error: 1.0991 - val_loss: 0.7906 - val_root_mean_squared_error: 0.8892
Epoch 5/50
179/179 [===========

179/179 [==============================] - 37s 206ms/step - loss: 0.8446 - root_mean_squared_error: 0.9190 - val_loss: 0.7496 - val_root_mean_squared_error: 0.8658
Epoch 42/50
179/179 [==============================] - 37s 207ms/step - loss: 0.8473 - root_mean_squared_error: 0.9205 - val_loss: 0.7687 - val_root_mean_squared_error: 0.8768
Epoch 43/50
179/179 [==============================] - 37s 206ms/step - loss: 0.8244 - root_mean_squared_error: 0.9080 - val_loss: 0.7567 - val_root_mean_squared_error: 0.8699
Epoch 44/50
179/179 [==============================] - 37s 206ms/step - loss: 0.8299 - root_mean_squared_error: 0.9110 - val_loss: 0.7900 - val_root_mean_squared_error: 0.8888
Epoch 45/50
179/179 [==============================] - 37s 206ms/step - loss: 0.8454 - root_mean_squared_error: 0.9194 - val_loss: 0.8219 - val_root_mean_squared_error: 0.9066
Epoch 46/50
179/179 [==============================] - 37s 207ms/step - loss: 0.8202 - root_mean_squared_error: 0.9057 - val_loss: 0

In [17]:
hist_df = pd.DataFrame(history.history) 
hist_csv_file = 'using-standard-albert-history-01.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [18]:
print(time_callback.times)

[46.761653900146484, 35.9401216506958, 36.51329731941223, 36.73858451843262, 36.960084199905396, 36.98572087287903, 36.99850606918335, 37.01041531562805, 37.1099169254303, 37.07095503807068, 37.00225281715393, 36.95877122879028, 37.02801275253296, 36.99090909957886, 36.97313165664673, 36.938053607940674, 37.038631200790405, 37.019874811172485, 36.92543983459473, 36.939226388931274, 37.030810594558716, 36.968754529953, 36.92278075218201, 36.888179302215576, 36.985438108444214, 36.994746685028076, 36.94369101524353, 36.87661361694336, 36.920745849609375, 36.964438676834106, 36.934975385665894, 36.85775637626648, 36.94272327423096, 36.99184441566467, 36.88790559768677, 36.859930753707886, 36.91948413848877, 36.979132413864136, 36.89594578742981, 36.90595197677612, 36.88626432418823, 36.99145030975342, 36.88920617103577, 36.873992919921875, 36.85991382598877, 37.00477862358093, 36.915162086486816, 36.91175985336304, 36.85884118080139, 36.94232678413391]


In [19]:
hist_time_df = pd.DataFrame(time_callback.times) 
hist_csv_file = 'using-standard-albert-time-01.csv'
with open(hist_csv_file, mode='w') as f:
    hist_time_df.to_csv(f)

In [20]:
model.save_weights('./using-standard-albert-model-01.h5')

# Evaluation

In [21]:
model.evaluate(test_data, verbose=1)

43/43 [==============================] - 7s 164ms/step - loss: 0.9189 - root_mean_squared_error: 0.9586


[0.9189221858978271, 0.9586042761802673]

# Testing

In [22]:
def check_similarity(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    data = AlbertSemanticDataGenerator(
        sentence_pairs, scores=None, batch_size=1, shuffle=False, include_targets=False,
    )

    score = model.predict(data)[0]
    return score

In [23]:
sentence1 = 'A distributed system is a system from multiple computers which communicate with each other via network'
sentence2 = 'A distributed system is a system from one computer without network'
check_similarity(sentence1, sentence2)

array([4.3049545], dtype=float32)

In [24]:
sentence1 = 'asdsd'
sentence2 = 'asdsad are good sdsdsdsds'
check_similarity(sentence1, sentence2)

array([0.91901344], dtype=float32)

In [25]:
sentence1 = 'the activity of learning or being trained'
sentence2 = 'the gradual process of acquiring knowledge.'
check_similarity(sentence1, sentence2)

array([1.6289066], dtype=float32)